# TensorflowとKerasで畳み込みニューラルネットワークでディープラーニング
tensorflowとkerasを用いてmnistという手書き文字のデータを学習させる。畳み込みニューラルネットワークをkerasで組み、学習させて正確性を検証する。

## 1. 必要なモジュールのimport
      
今回は多層パーセプトロンの時に読み込んだモジュールに加え、畳み込みニューラルネットワークを組むためにkeras.layersからFlatten, Conv2D, MaxPooling2Dを追加で読み込む。

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import SGD 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## 2.MNISTデータの読み込み
          
mnist.load_data()で60,000枚の28x28，10個の数字の白黒画像と10,000枚のテスト用画像データセットを読み込む。前回同様。    
      

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

        
## 3.読み込んだデータを畳み込みニューラルネットワークで学習するための形に変形する
          
畳み込みニューラルネットワーク(CNN, Convolutional Neural Network)では、データの2次元構造を保持したままニューラルネットワークにデータを入力する。読み込んだMNISTを適した形に変形する。    
    
具体的には、.reshapeを用いて(枚数、28, 28, チャンネル数)の形にする。チャンネル数は白黒画像であれば1、カラー画像であれば3となる。

In [3]:
X_train = X_train.reshape(X_train.shape[0],28,28,1).astype('float32')
X_test = X_test.reshape(X_test.shape[0],28,28,1).astype('float32')
X_train /= 255.0
X_test /= 255.0
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

## 4.Kerasを用いてニューラルネットを組む
    
Kerasを用いて畳み込みニューラルネットワークを組む。  
    
畳み込みニューラルネットワークでは、一定の大きさのフィルター(重みの集まり)を少しずつずらしながら画像に当てていき、計算を進めていく。   
    
畳み込みの計算では、下記のようにフィルターを当てたときのそれぞれの数値の積の合計を計算して出力する。

<img src="conv2d.png" width="600" height="600" >

Max-poolingの計算では、下記のように決められたサイズの中の最大値を出力する。

<img src="max_pooling.png" width="400" height="400" >

実際に作成した下記のニューラルネットワークでは、最初の畳み込み層(Conv2D)は28x28ピクセルの画像に3x3ピクセルのフィルターをずらしながら当てていくことで26x26ピクセルのデータが出力される。ここの畳み込み層では16枚のフィルターを準備して、26x26ピクセルのデータを16枚出力することになる。

MaxPooling2Dでは定められた範囲内のうち最大のピクセルのみを選んで残す。下記のConv2Dの次のMaxPooling2Dでは26x26ピクセルのデータが13x13ピクセルのデータになることになる。出力されるのは13x13ピクセルのデータが16枚ということになる。    
     
次の畳み込み層では再び3x3ピクセルのフィルターを当てていき、13x13ピクセルのデータは11x11ピクセルとなる。ここの畳み込み層では32枚のフィルターを当てるため、11x11ピクセルのデータを32枚出力することになる。その次のMaxPooling2Dで5x5ピクセルのデータが32枚出力される。

flattenによって全ての数値を一次元に並べる。flattenによって5x5x32 = 800 個のunitが並んでいる状態となる。

その次のDense層で800個のunitは256個のunitへ伝達され、最終的に10個のunitへ入力されてsoftmaxで出力される。

In [4]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

上のニューラルネットワークを図示すると下のようになる。

<img src="cnn.png" width="800" height="800" >

## 5.作成したニューラルネットをどのように学習させるかを設定する
上のプログラムで作成したニューラルネットを、どのように学習させるか設定する。    
    
具体的には.compile()で損失関数(loss)、最適化アルゴリズム(optimizer)、評価関数(metrics)を与えてニューラルネットを完成させる。    
    
一般的には分類問題であればlossはcategorical_crossentropyを用いる。    
    
oprimizerは下記では確率的勾配降下法（SGD, Stochastic Gradient Descent）を用いており、lrはlearning rateを示す。learning rateが大きいほど、一回の学習でのパラメータの更新は大きくなる。    

分類問題であるため、評価関数はaccuracyを用いる。

In [5]:
model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01), metrics=["accuracy"])

## 6.学習
.fit関数を用いて学習データ、正解データ、エポック数、検証用に用いるデータの割合、バッチサイズを与えて学習を開始する。    

In [6]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=100)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 13s - loss: 1.2713 - acc: 0.6562 - val_loss: 0.3736 - val_acc: 0.8898

## 7.テストデータを用いて学習結果を検証する
あらかじめとっておいたテスト用データ(X_test, y_test)を用いて、今学習させたニューラルネットワークの正確性を検証する。    
.predictでテスト用データを与え、予測結果を出力する。    
    
y_pred（モデルによる予測）とy_test（正解データ）から精度を算出する。

In [ ]:
y_pred = model.predict(X_test, batch_size=100)
y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import f1_score
print('accuracy:',f1_score(Y_test, y_pred, average='macro'))

## 8.画像を用いて予測結果の検証(参考)
    
MNISTのX_testを、正解と予測結果と合わせて可視化して結果を眺める。多層パーセプトロンの時よりも良くなっていそうなのがわかる。

In [ ]:
fig = plt.figure(figsize=(12, 50))

for i in range(100):
    plt.subplot(20, 5, i+1)
    plt.imshow((X_test[i]* 255).astype(np.int32).reshape(28, 28), cmap='gray')
    if Y_test[i] == y_pred[i]:
        plt.title("No.{0} / Answer:{1}, Predict:{2}".format(i, Y_test[i], y_pred[i]))
    else:
        plt.title("No.{0} / Answer:{1}, Predict:{2}".format(i, Y_test[i], y_pred[i]), color="red")
    plt.axis("off")
plt.tight_layout()

## 9.ネットワークを変えてやってみる
kerasによるニューラルネットワーク構築部分を変えて、精度が改善しないか試行する。

In [ ]:
model = Sequential()

#構造をここに記入

model.add(Dense(10, activation='softmax'))

In [ ]:
#モデルをコンパイルして学習させてみる
model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01), metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=100)

In [ ]:
#最終的な精度を計算する
y_pred = model.predict(X_test, batch_size=100)
y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import f1_score
print('accuracy:',f1_score(Y_test, y_pred, average='macro'))